# Construindo uma Antomação Web

In [1]:
# Obtendo as bibliotecas e verificando sua versão
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime as dt
import time
# Verificando as versões 
print('Versão do Pandas:', pd.__version__) 
print('Versão do Selenium:', selenium.__version__)



Versão do Pandas: 2.0.3
Versão do Selenium: 4.15.2


# 1. Obtendo as Cotações Atualizadas

In [2]:
# Etapa 1: abrir um navegador
navegador = webdriver.Chrome()

# Etapa 2: Obtendo o Xpath
# Obtendo o xpath da página de busca caixa de busca do Google
caixa_texto_busca = '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/div/textarea'
# Como os objetos são iguais obtemos o #path do obejto buscado 
objeto_buscado = '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'

# Etapa 3: Obtendo as cotações online:
# Dólar
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH, caixa_texto_busca).send_keys("cotação dólar")
navegador.find_element(By.XPATH,caixa_texto_busca ).send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH, objeto_buscado).get_attribute("data-value") 
# Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH, caixa_texto_busca).send_keys("cotação euro")
navegador.find_element(By.XPATH, caixa_texto_busca).send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, objeto_buscado).get_attribute("data-value")

# Visualizando as Variáveis
print('Cotação do Euro : ', cotacao_euro)
print('Cotação do Dolar: ', cotacao_dolar)

# Etapa 4: Fechando o navegador
navegador.quit()

Cotação do Euro :  5.431254444
Cotação do Dolar:  4.989599999999999


# 2. Contruindo uma tabela para executar uma atualização

Supondo que temos uma base de dados que necessitamos atualizar o preço de aquisição e o de venda, considerando ainda a margem de lucro sobre o preço do produto.


In [3]:
# Etapa 5 : Construindo uma tabela para aplicar correção
tabela = pd.DataFrame({'Cod': [1,2,3,4,5],
                       'Produto': ['TV Plasma', 'Xbox S', 'PS5', 'Notebook', 'Violão'],
                       'Valor Original': [1000, 800, 900, 750, 300],
                       'Moedas': ['Dólar', 'Euro', 'Euro', 'Dólar', 'Euro'],
                       'Cotação': [1, 1, 1, 1, 1],
                       'Valor Compra': [1000, 800, 900, 600, 300],
                       'Margem': [1.5, 1.5, 1.5,  1.5, 1.5],
                       'Valor Venda': [1500, 1200, 1350, 900, 450], 
                       'Data Atualização': ["01/01/2023", "01/01/2023", "01/01/2023", "01/01/2023", "01/01/2023"]
                       })

# Visualizando a tabela
tabela

,Cod,Produto,Valor Original,Moedas,Cotação,Valor Compra,Margem,Valor Venda,Data Atualização
0,1,TV Plasma,1000,Dólar,1,1000,1.5,1500,01/01/2023
1,2,Xbox S,800,Euro,1,800,1.5,1200,01/01/2023
2,3,PS5,900,Euro,1,900,1.5,1350,01/01/2023
3,4,Notebook,750,Dólar,1,600,1.5,900,01/01/2023
4,5,Violão,300,Euro,1,300,1.5,450,01/01/2023


# 3. Executando a Atualização da Tabela Original pelas Novas Cotações

Aplicando as Atualização da cotação e recalculando **Valor Compra** e **Valor Venda.**

In [4]:
# Etapa 6: Aplicano a atualização da coluna Moedas
# Atualizando data
tabela['Data Atualização'] = dt.now().strftime("%d/%m/%Y %H:%M")

# Atualizando moedas
tabela.loc[tabela['Moedas']=='Dólar', 'Cotação']= float(cotacao_dolar)
tabela.loc[tabela['Moedas']=='Euro', 'Cotação']= float(cotacao_euro)

# Etapa 7:  Corrigindo os valores de Valor Compra e Valor Venda.
tabela['Valor Compra'] = round(tabela['Valor Original'] * tabela['Cotação'],2)
tabela['Valor Venda'] = round(tabela['Valor Compra'] * tabela['Margem'] ,2)

# Visualizando a tabela atualizada
tabela

,Cod,Produto,Valor Original,Moedas,Cotação,Valor Compra,Margem,Valor Venda,Data Atualização
0,1,TV Plasma,1000,Dólar,4.989600,4989.60,1.5,7484.40,23/01/2024 07:24
1,2,Xbox S,800,Euro,5.431254,4345.00,1.5,6517.50,23/01/2024 07:24
2,3,PS5,900,Euro,5.431254,4888.13,1.5,7332.20,23/01/2024 07:24
3,4,Notebook,750,Dólar,4.989600,3742.20,1.5,5613.30,23/01/2024 07:24
4,5,Violão,300,Euro,5.431254,1629.38,1.5,2444.07,23/01/2024 07:24
